<a href="https://colab.research.google.com/github/abhidendukuri/Sports_Modeling/blob/HW8/DS3001_HW8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pylogit

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pylogit
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error, brier_score_loss

from scipy.special import logit
from tqdm import tqdm
pd.options.display.max_columns = 100

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def mlogit(formula, df, obs_id, alt_id) :
    """
    df must be sorted by obs_id to use this function
    """
    from patsy import dmatrices
    from collections import OrderedDict
    import pylogit as pl
    import numpy as np
    data = dmatrices(formula+' -1', df, return_type = "dataframe")
    all_alts = sorted(df.loc[:,alt_id].unique())
    
    spec = OrderedDict()
    names = OrderedDict()
    spec['intercept'] = all_alts[1:]
    names['intercept'] = [f'(Intercept):{alt}' for alt in all_alts[1:]]
    c_names = data[1].design_info.column_names
    choice = data[0].design_info.column_names[0]
    data[1][choice] = data[0]
    data[1][[obs_id,alt_id]] = df[[obs_id,alt_id]]    
    for c in c_names :
        spec[c] = [all_alts]
        names[c] = [c]
    model = pl.create_choice_model(data = data[1], alt_id_col = alt_id, obs_id_col = obs_id, 
                                   choice_col = choice, specification = spec, names = names, model_type = 'MNL')
    model.fit_mle(np.zeros(model.design.shape[1]), method = 'Powell')
    return model

In [3]:
df = pd.read_csv('xc.csv', parse_dates = ['dt'])
df = df.rename(columns = {'dt' : 'date'})
df = df.loc[df.race_type == 'flat_race']
df = df.sort_values('date')

In [4]:
win_choice = df.loc[df.winner == 1].sample(frac = 1.0).groupby('race_id').head(1).loc[:,['race_id','box']].copy()
win_choice['twinner'] = 1
df = df.merge(win_choice, on = ['race_id','box'], how = 'left')
df.twinner = df.twinner.fillna(0)

df = df.sort_values(['race_id','box'])
df = df[df.race_type == 'flat_race']

In [5]:
startupTime = 1.75
distDft = 400
distExp = 0.11
alpha = 0.03

halflife = pd.Timedelta(-np.log(2)/np.log(1-alpha), unit = 'Days')
print(halflife, (1-alpha)**(halflife/pd.Timedelta(1, unit = 'Days')))

22 days 18:09:27.912643200 0.4999999999965491


In [6]:
df['mmps'] = df.distance_m/(df.time-startupTime)*(df.distance_m/distDft)**distExp
df['avg_mmps'] = df.groupby('dog_id').mmps.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df['ema_mmps'] = df.groupby('dog_id').apply(
    lambda x : x.mmps.ewm(halflife=halflife,times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id',drop = True)

In [7]:
df['prior_races'] = df.groupby(['dog_id']).mmps.transform(lambda x : x.expanding().count().shift(1, fill_value = 0))
dt_prev = df.groupby(['dog_id']).date.transform(lambda x : x.shift(1, fill_value = np.NaN))
df['dt_prev'] = 1*(df.date - dt_prev <= '90 days')
df['drop_race'] = 1*((df.dt_prev == 0) | (df.prior_races < 3))
df2 = df[~df.race_id.map(df.groupby('race_id').sum().drop_race > 0)]
df2 = df2.dropna()

In [8]:
train = df2[df2.date.between('2019-07-01', '2020-01-31')]
test = df2[df2.date >= '2020-02-01']

In [9]:
pylogit.choice_calcs.__file__

'/usr/local/lib/python3.7/dist-packages/pylogit/choice_calcs.py'

# 1.

In [10]:
model = mlogit('twinner ~ avg_mmps', train, 'race_id', 'box')
model.get_statsmodels_summary()

Log-likelihood at zero: -34,859.3900
Initial Log-likelihood: -34,859.3900


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method Powell does not use gradient information (jac).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method Powell does not use Hessian information (hess).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/pylogit/estimation.py:686: OptimizeWarning: Unknown solver options: gtol
  **kwargs)


Estimation Time for Point Estimation: 8.59 seconds.
Final log-likelihood: -34,479.9548


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               19,877
Model:             Multinomial Logit Model   Df Residuals:                   19,871
Method:                                MLE   Df Model:                            6
Date:                     Tue, 11 May 2021   Pseudo R-squ.:                   0.011
Time:                             15:23:21   Pseudo R-bar-squ.:               0.011
AIC:                            68,971.910   Log-Likelihood:            -34,479.955
BIC:                            69,019.294   LL-Null:                   -34,859.390
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0438      0.025     -1.778      0.075      -0.092       0.004
(Intercept):3     0.0193      0.024      0.793      0.428      -0.028       0.067
(Intercept):4    -0.0150      0.025     -0.609      0.542      -0.063       0.033
(Intercept):5    -0.1073      0.025     -4.277      0.000      -0.156      -0.058
(Intercept):6     0.0114      0.024      0.465      0.642      -0.037       0.059
avg_mmps          1.2638      0.048     26.555      0.000       1.171       1.357
=================================================================================
"""

In [11]:
y_pred = model.predict(test)
brier_score_loss(test.twinner, y_pred)

/usr/local/lib/python3.7/dist-packages/pylogit/base_multinomial_cm_v2.py:381: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["intercept"] = 1.0


0.14061682987350585

# 2.

## a.


In [12]:
train_2 = df2[df2.date.between('2019-07-01','2019-11-30')]
test_2 = df2[df2.date.between('2019-12-01', '2020-01-31')]

model = smf.ols('mmps ~ ema_mmps', data = train_2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.758
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                 2.469e+05
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        15:23:21   Log-Likelihood:                -6561.1
No. Observations:               78634   AIC:                         1.313e+04
Df Residuals:                   78632   BIC:                         1.314e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8223      0.033     24.691      0.000       0.757       0.888
ema_mmps       0.9530      0.002    496.869      0.000       0.949       0.957
==============================================================================
Omnibus:                     6909.391   Durbin-Watson:                   1.360
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24137.597
Skew:                          -0.420   Prob(JB):                         0.00
Kurtosis:                       5.581   Cond. No.                         619.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
y_pred = model.predict(test_2.drop('mmps',1))
mean_squared_error(test_2.mmps, y_pred)

0.06873040620569776

## b.

In [14]:
for sid in tqdm(sorted(df2.stadium_id.unique())):
  a = 'a' + str(sid)
  b = 'b' + str(sid)
  df2[a] = 1*(df2.stadium_id == sid)
  df2[b] = df2.groupby('dog_id').apply(
    lambda x : x[a].ewm(halflife=halflife, times = x.date).mean().shift(1, fill_value=0)
).reset_index(level = 'dog_id', drop = True)

for sid in sorted(df2.stadium_id.unique()):
  i = 'i' + str(sid)
  a = 'a' + str(sid)
  b = 'b' + str(sid)
  
  df2[i] = df2[a] - df2[b]

100%|██████████| 21/21 [11:21<00:00, 32.44s/it]


In [15]:
train_2 = df2[df2.date.between('2019-07-01','2019-11-30')]
test_2 = df2[df2.date.between('2019-12-01', '2020-01-31')]

stadium_id = ' + '.join(df2.columns[-21:])

f = 'mmps ~ ema_mmps + ' + stadium_id
model = smf.ols(f, data = train_2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.784
Model:                            OLS   Adj. R-squared:                  0.784
Method:                 Least Squares   F-statistic:                 1.294e+04
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        15:34:43   Log-Likelihood:                -2235.8
No. Observations:               78634   AIC:                             4518.
Df Residuals:                   78611   BIC:                             4731.
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0219      0.032     32.283      0.000       0.960       1.084
ema_mmps       0.9413      0.002    516.399      0.000       0.938       0.945
i13003         0.0877      0.015      5.970      0.000       0.059       0.117
i13004         0.0838      0.011      7.353      0.000       0.061       0.106
i13007        -0.2217      0.010    -21.384      0.000      -0.242      -0.201
i13008        -0.0141      0.015     -0.920      0.357      -0.044       0.016
i13009        -0.0712      0.009     -7.882      0.000      -0.089      -0.054
i13010         0.3991      0.010     38.140      0.000       0.379       0.420
i13013         0.1243      0.019      6.598      0.000       0.087       0.161
i13014         0.0825      0.013      6.272      0.000       0.057       0.108
i13019         0.0439      0.010      4.561      0.000       0.025       0.063
i13020         0.0012      0.015      0.081      0.935      -0.029       0.031
i13021         0.0151      0.043      0.350      0.726      -0.069       0.099
i13023         0.1025      0.014      7.327      0.000       0.075       0.130
i13025         0.4453      0.010     43.201      0.000       0.425       0.466
i13026        -0.1618      0.010    -16.897      0.000      -0.181      -0.143
i13035         0.1772      0.016     11.290      0.000       0.146       0.208
i13037         0.2322      0.010     23.770      0.000       0.213       0.251
i13043         0.0697      0.012      6.016      0.000       0.047       0.092
i13048        -0.4168      0.011    -38.122      0.000      -0.438      -0.395
i13053         0.0340      0.020      1.711      0.087      -0.005       0.073
i13059         0.1144      0.021      5.423      0.000       0.073       0.156
i13061        -0.3863      0.012    -31.739      0.000      -0.410      -0.362
==============================================================================
Omnibus:                     5075.159   Durbin-Watson:                   1.396
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            11118.431
Skew:                          -0.428   Prob(JB):                         0.00
Kurtosis:                       4.632   Cond. No.                         844.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
y_pred = model.predict(test_2.drop('mmps',1))
mean_squared_error(test_2.mmps, y_pred)

0.06479001360980052

In [17]:
df2.to_csv('xc2.csv', index=False)

# 3.

In [18]:
df3 = pd.read_csv('xc2.csv', parse_dates=['date'])
df3['comment'] = df3.comment.fillna('')

In [19]:
race_comments = [
    'QAw|QAway|QuickAway',
    'SAw|SAway|SlowAway',
    'MsdBrk|MsdBreak|MissedBreak',
    'TurnedInTrap',
    'Disp|DispLd',
    'NvShw',
    'CmAg',
    'ClrRun|ClearRun',
    'HldOn|HeldOn',
    'FinWll|FinWell|FinishedWell',
    'Railed',
    'Bmp|Bumped',
    'Blk|Baulk|Baulked',
    'Crd|Crowded|CrD',
    'Imp|Impeded',
    'StruckInto',
    'Ck|Chk|Checked',
    'Stb|Stumbled',
    'KO',
    'EP|EarlyPace|EPace|Ep|EarlyP',
    'LckEP|LackedEP',
    'Ld|Led',
    'ALd|ALed|AlwaysLed',
    'LdNrLn|LdNrLine|LedNearLine',
    'LedToNearLine',
    'W|Wide',
    'VW|VWide|VeryWide',
    'Badly',
    'VB|VBadly',
    'Fcd|Forced'
]


In [20]:
for cmt in tqdm(race_comments):
  a = 'a' + cmt.split('|')[0]
  b = 'b' + cmt.split('|')[0]
  df3[a] = 1*(df3.comment.str.contains(cmt))
  df3[a] = df3.groupby('dog_id')[a].shift(1)
  df3[b] = df3.groupby('dog_id').apply(
    lambda x: x[a].ewm(halflife=halflife, times = x.date).mean()
).reset_index(level = 'dog_id', drop = True)

df3 = df3.dropna()
for cmt in race_comments:
  i = 'i' + cmt.split('|')[0]
  a = 'a' + cmt.split('|')[0]
  b = 'b' + cmt.split('|')[0]
  
  df3[i] = df3[a] - df3[b]

100%|██████████| 30/30 [15:04<00:00, 30.15s/it]


## a.

In [21]:
train_2 = df3[df3.date.between('2019-07-01','2019-11-30')]
test_2 = df3[df3.date.between('2019-12-01', '2020-01-31')]

indicators = ' + '.join(df3.columns[df3.columns.str.startswith('i')])

f = 'mmps ~ ema_mmps + ' + indicators
model = smf.ols(f, data = train_2).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   mmps   R-squared:                       0.795
Model:                            OLS   Adj. R-squared:                  0.794
Method:                 Least Squares   F-statistic:                     5719.
Date:                Tue, 11 May 2021   Prob (F-statistic):               0.00
Time:                        15:53:39   Log-Likelihood:                -135.91
No. Observations:               75494   AIC:                             375.8
Df Residuals:                   75442   BIC:                             855.9
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.7061      0.031     22.444      0.000       0.644       0.768
ema_mmps           0.9595      0.002    529.524      0.000       0.956       0.963
i13003            -0.0394      0.017     -2.365      0.018      -0.072      -0.007
i13004             0.1070      0.012      8.825      0.000       0.083       0.131
i13007            -0.4453      0.012    -36.217      0.000      -0.469      -0.421
i13008             0.0246      0.023      1.087      0.277      -0.020       0.069
i13009            -0.1908      0.010    -18.641      0.000      -0.211      -0.171
i13010             0.4940      0.012     42.009      0.000       0.471       0.517
i13013             0.1775      0.027      6.570      0.000       0.125       0.230
i13014             0.1378      0.017      8.032      0.000       0.104       0.171
i13019            -0.0730      0.011     -6.391      0.000      -0.095      -0.051
i13020            -0.1216      0.018     -6.917      0.000      -0.156      -0.087
i13021            -0.0329      0.054     -0.610      0.542      -0.139       0.073
i13023             0.1047      0.016      6.376      0.000       0.073       0.137
i13025             0.4528      0.012     38.168      0.000       0.430       0.476
i13026            -0.3221      0.011    -28.987      0.000      -0.344      -0.300
i13035             0.2358      0.019     12.356      0.000       0.198       0.273
i13037             0.2901      0.012     24.976      0.000       0.267       0.313
i13043             0.0697      0.014      5.101      0.000       0.043       0.097
i13048            -0.6694      0.012    -54.361      0.000      -0.694      -0.645
i13053             0.1014      0.032      3.126      0.002       0.038       0.165
i13059             0.1825      0.027      6.643      0.000       0.129       0.236
i13061            -0.4835      0.013    -36.779      0.000      -0.509      -0.458
iQAw              -0.0079      0.005     -1.478      0.139      -0.018       0.003
iSAw           -5.732e-05      0.004     -0.014      0.989      -0.008       0.008
iMsdBrk            0.0091      0.008      1.157      0.247      -0.006       0.024
iTurnedInTrap      0.2735      0.395      0.692      0.489      -0.502       1.048
iDisp             -0.0026      0.016     -0.164      0.869      -0.033       0.028
iNvShw             0.3401      0.129      2.639      0.008       0.088       0.593
iCmAg              0.0104      0.022      0.477      0.634      -0.032       0.053
iClrRun           -0.0046      0.006     -0.736      0.462      -0.017       0.008
iHldOn            -0.0041      0.021     -0.194      0.846      -0.045       0.037
iFinWll            0.0138      0.016      0.870      0.384      -0.017       0.045
iRailed            0.0046      0.024      0.192      0.848      -0.042       0.051
iBmp        

## b.

In [22]:
y_pred = model.predict(test_2.drop('mmps',1))
mean_squared_error(test_2.mmps, y_pred)

0.06283450130513628

In [23]:
df3.to_csv('xc3.csv', index=False)

# 4.

## a.

In [24]:
df4 = pd.read_csv('xc3.csv', parse_dates=['date'])
df4['mmps_pred'] = model.predict(df4)
train_4 = df4[df4.date < '2020-02-01']
test_4 = df4[df4.date >= '2020-02-01']

In [25]:
model = mlogit('twinner ~ mmps_pred', train_4, 'race_id', 'box')
model.get_statsmodels_summary()

Log-likelihood at zero: -51,357.1101
Initial Log-likelihood: -51,357.1101


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method Powell does not use gradient information (jac).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method Powell does not use Hessian information (hess).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/pylogit/estimation.py:686: OptimizeWarning: Unknown solver options: gtol
  **kwargs)


Estimation Time for Point Estimation: 15.79 seconds.
Final log-likelihood: -50,091.5004


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               32,682
Model:             Multinomial Logit Model   Df Residuals:                   32,676
Method:                                MLE   Df Model:                            6
Date:                     Tue, 11 May 2021   Pseudo R-squ.:                   0.025
Time:                             16:00:49   Pseudo R-bar-squ.:               0.025
AIC:                           100,195.001   Log-Likelihood:            -50,091.500
BIC:                           100,245.368   LL-Null:                   -51,357.110
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
(Intercept):2    -0.0431      0.020     -2.199      0.028      -0.082      -0.005
(Intercept):3     0.0436      0.019      2.245      0.025       0.006       0.082
(Intercept):4     0.0013      0.020      0.068      0.945      -0.037       0.040
(Intercept):5    -0.1241      0.020     -6.220      0.000      -0.163      -0.085
(Intercept):6     0.0039      0.019      0.201      0.841      -0.034       0.042
mmps_pred         2.8236      0.057     49.489      0.000       2.712       2.935
=================================================================================
"""

In [26]:
y_pred = model.predict(test_4.drop('twinner', 1))
brier_score_loss(test_4['twinner'], y_pred)

0.13894176040798398

## b.

In [64]:
df4['going_prev'] = df4.groupby('dog_id').going.transform(lambda x : x.expanding().mean().shift(1, fill_value = 0))
df4['decimal_price_prob'] = df4.decimal_price / (df4.decimal_price + 1)
df4['decimal_price_prev'] = df4.groupby('dog_id').decimal_price_prob.transform(lambda x : x.expanding().mean().shift(1))

In [66]:
df4 = df4.dropna()

In [67]:
train_4 = df4[df4.date < '2020-02-01']
test_4 = df4[df4.date >= '2020-02-01']

In [68]:
model = mlogit('twinner ~ mmps_pred + distance_m + going_prev + decimal_price_prev', train_4, 'race_id', 'box')
model.get_statsmodels_summary()

Log-likelihood at zero: -45,678.2921
Initial Log-likelihood: -45,678.2921


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method Powell does not use gradient information (jac).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method Powell does not use Hessian information (hess).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/pylogit/estimation.py:686: OptimizeWarning: Unknown solver options: gtol
  **kwargs)


Estimation Time for Point Estimation: 21.40 seconds.
Final log-likelihood: -44,318.3010


/usr/local/lib/python3.7/dist-packages/pylogit/base_multinomial_cm_v2.py:1228: RuntimeWarning: invalid value encountered in sqrt
  self._store_inferential_results(np.sqrt(np.diag(self.cov)),


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               31,992
Model:             Multinomial Logit Model   Df Residuals:                   31,983
Method:                                MLE   Df Model:                            9
Date:                     Tue, 11 May 2021   Pseudo R-squ.:                   0.030
Time:                             17:27:40   Pseudo R-bar-squ.:               0.030
AIC:                            88,654.602   Log-Likelihood:            -44,318.301
BIC:                            88,729.961   LL-Null:                   -45,678.292
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
(Intercept):2         -0.0573      0.020     -2.838      0.005      -0.097      -0.018
(Intercept):3          0.0383      0.020      1.915      0.055      -0.001       0.078
(Intercept):4          0.0006      0.020      0.030      0.976      -0.039       0.040
(Intercept):5         -0.1130      0.020     -5.531      0.000      -0.153      -0.073
(Intercept):6          0.0211      0.020      1.069      0.285      -0.018       0.060
mmps_pred              2.7686      0.005    508.047      0.000       2.758       2.779
distance_m             0.0401        nan        nan        nan         nan         nan
going_prev            -0.1806      0.093     -1.944      0.052      -0.363       0.001
decimal_price_prev    -3.5815      0.168    -21.268      0.000      -3.912      -3.251
======================================================================================
"""

In [69]:
y_pred = model.predict(test_4.drop('twinner', 1))
brier_score_loss(test_4['twinner'], y_pred)

0.1379358197967272

## c.

In [84]:
from scipy.special import logit

df4['twin_pred'] = model.predict(df4)
df4['twin_pred'] = np.where(df4.twin_pred == 1, df4.twin_pred - 0.00000000001, df4.twin_pred)
df4['logit_twin_pred'] = logit(df4.twin_pred)
df4['logit_dml_price'] = logit(df4.decimal_price_prob)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
train_4 = df4[df4.date < '2020-02-01']
test_4 = df4[df4.date >= '2020-02-01']

In [97]:
model = mlogit('twinner ~ logit_twin_pred + logit_dml_price + mmps_pred + distance_m + going_prev', train_4, 'race_id', 'box')
model.get_statsmodels_summary()

Log-likelihood at zero: -45,678.2921
Initial Log-likelihood: -45,678.2921


/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:506: RuntimeWarning: Method Powell does not use gradient information (jac).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/_minimize.py:511: RuntimeWarning: Method Powell does not use Hessian information (hess).
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/pylogit/estimation.py:686: OptimizeWarning: Unknown solver options: gtol
  **kwargs)


Estimation Time for Point Estimation: 1.12 minutes.
Final log-likelihood: -44,329.0428


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Multinomial Logit Model Regression Results                    
===================================================================================
Dep. Variable:                     twinner   No. Observations:               31,992
Model:             Multinomial Logit Model   Df Residuals:                   31,982
Method:                                MLE   Df Model:                           10
Date:                     Tue, 11 May 2021   Pseudo R-squ.:                   0.030
Time:                             17:55:53   Pseudo R-bar-squ.:               0.029
AIC:                            88,678.086   Log-Likelihood:            -44,329.043
BIC:                            88,761.818   LL-Null:                   -45,678.292
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
(Intercept):2      -0.0551      0.021     -2.668      0.008      -0.096      -0.015
(Intercept):3       0.0361      0.020      1.791      0.073      -0.003       0.076
(Intercept):4       0.0002      0.020      0.012      0.990      -0.039       0.040
(Intercept):5      -0.1092      0.022     -4.965      0.000      -0.152      -0.066
(Intercept):6       0.0191      0.020      0.967      0.333      -0.020       0.058
logit_twin_pred     0.0290      0.054      0.537      0.591      -0.077       0.135
logit_dml_price    -0.4904      0.045    -10.822      0.000      -0.579      -0.402
mmps_pred           2.6764      0.205     13.026      0.000       2.274       3.079
distance_m         -0.2332    457.947     -0.001      1.000    -897.792     897.326
going_prev         -0.1705      0.094     -1.811      0.070      -0.355       0.014
===================================================================================
"""

In [98]:
y_pred = model.predict(test_4)
brier_score_loss(test_4['twinner'], y_pred)

0.13798327766307228